# Imported needed libraries

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np 
import imutils
import time 
import cv2 as cv 
import os 

# **Create Function `detect_and_predict_mask`**

In [3]:
def detect_and_predict_mask(frame,faceNet,maskNet):
    #grab the dimensions of the frame and then construct a blob from it
    (height,width) = frame.shape[:2]#Return height and width from each frame 
    blob = cv.dnn.blobFromImage(frame,1.0,(224,224),(104.0,177.0,123.0))#Mean for imageNet RGB
    # Pass the blob through the network and obtain the face detections 
    faceNet.setInput(blob)
    detections = faceNet.forward()#Return face 
    print("The detections of shape are ==> ",detections.shape)
    #Initialize our list of faces , their crosponding locations , 
    # and the list of predictions from face mask network 
    faces =  []
    locs  =  []
    preds =  []
    # Loop Over the detections 
    for i in range (0, detections.shape[2]):
        # Extract the confidence (i.e , probability associated with the detection)
        confidence = detections[0,0,i,2]
        #Filter out weak detections by ensuring the confidence is greater than a minmum confidence 
        if confidence>0.5:
            #Compute the (x,y) coordinates of the bounding box for the object 
            box = detections[0,0,i,3:7] * np.array([width,height,width,height])
            (start_x,start_y,end_x,end_y) = box.astype("int")
            #Ensure the bounding boxes fall within the dimesions of the frame 
            (start_x,start_y)= (max(0,start_x),max(0,start_y))
            (end_x,end_y)=(min(width - 1,end_x), min (height - 1 , end_y))
            #Extract the face ROI , Convert it from BGR To RGB Channels 
            #Ordering , resize it to 224 * 224 , and preprocess it 
            face = frame[start_y : end_y , start_x:end_x]
            #face = cv.cvtColor(face,cv.COLOR_BGR2RGB)
            face = cv.resize(face,(224,224))
            face = img_to_array(face)
            face =preprocess_input(face)
            #Add the face and bounding boxes to their respective Lists 
            faces.append(face)
            locs.append((start_x,start_y,end_x,end_y))
    #Only Make a predictions if at least one face was detectd 
    if len (faces)>0:
    # For Faster inference we'll make batch predictions on "all"
    # Faces at the same time rather than one-by-one predictions
        # In the above for "loop"
          faces = np.array(faces,dtype="float32")
          preds =  maskNet.predict(faces,batch_size=32)
    # Returns a 2-tuple of the face locations and thier corresponding locations
    return (locs , preds)
     

# **Load Our Serialized face detector model from disk**

In [4]:
prototxtPath = r"D:\AI\AI Models\6. Models\1- PreTrained Model (MobileNet)\deploy.prototxt"
weightPath  = r"D:\AI\AI Models\6. Models\1- PreTrained Model (MobileNet)\MobileNetSSD_deploy.caffemodel"
faceNet = cv.dnn.readNet(prototxtPath,weightPath)
#Load The face mask detector model from disk 
maskNet = load_model ("mask_detector.model")

# **Initialize The Video Stream**

In [5]:
print('[INFO]....Starting Video Stream...')
video_stream = VideoStream(src=0).start()#Open Laptop or PC Camera
#Loop Over The Frame from video stream
while True:
    #Grab the frame from the threaded video stream and resize it 
    # To Have a maximum width of 400 pixels 
    frame =video_stream.read()
    frame = imutils.resize(frame,width = 400)#Resize frame and put width = 400 pixels for second 
    #detect faces in the frame and determine if they are wearing a face mask or Not
    (locations , predictions) = detect_and_predict_mask(frame,faceNet,maskNet)
    #Loop over the detected face locations and their corresponding loactions 
    for (box,pred) in zip (locations,predictions):
        # Unpack the bounding box and predictions 
        (startx,starty,endx,endy) = box
        (mask,withoutmask) = pred
        # Determine the class label and color we'll use to draw The Bounding box and text 
        label = "Mask" if mask > withoutmask else "No Mask"
        color = (0,255,0) if label == 'Mask' else (0,0,255)
        #Include the probability in the label 
        label = " {} : {:.2f}%".format(label,max(mask,withoutmask) * 100 )
        #display the label and bounding box rectangel on the output frame 
        cv.putText(frame,label,(startx,starty - 10) , cv.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv.rectangle(frame,(startx,starty),(endx,endy),color,2)
    # Show the output frame 
    cv.imshow("Frame",frame)
    key = cv.waitKey(1)
    if key == ord ('q'): break 
cv.destroyAllWindows()
video_stream.stop()

[INFO]....Starting Video Stream...
The detections of shape are ==>  (1, 1, 100, 7)
The detections of shape are ==>  (1, 1, 100, 7)
The detections of shape are ==>  (1, 1, 100, 7)
The detections of shape are ==>  (1, 1, 100, 7)
The detections of shape are ==>  (1, 1, 100, 7)
The detections of shape are ==>  (1, 1, 100, 7)
The detections of shape are ==>  (1, 1, 100, 7)
The detections of shape are ==>  (1, 1, 100, 7)
The detections of shape are ==>  (1, 1, 100, 7)
The detections of shape are ==>  (1, 1, 100, 7)
The detections of shape are ==>  (1, 1, 100, 7)
The detections of shape are ==>  (1, 1, 100, 7)
The detections of shape are ==>  (1, 1, 100, 7)
1/1 [==============================] - 1s 1s/step
The detections of shape are ==>  (1, 1, 100, 7)
The detections of shape are ==>  (1, 1, 100, 7)
1/1 [==============================] - 0s 47ms/step
The detections of shape are ==>  (1, 1, 100, 7)
The detections of shape are ==>  (1, 1, 100, 7)
The detections of shape are ==>  (1, 1, 100, 7)